District related pre-processing

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [9]:
date_dist_df = pd.read_csv('../../../data/main_data/fin_daily.csv')
districts_df = pd.read_csv('../../../data/main_data/districts.csv')
weather_df = pd.read_csv('../../../data/main_data/weather_pattern.csv')

In [10]:
print("date-district-dataset \n")
date_dist_df.head()
print("district-feature-dataset")
districts_df.head()
print("weather-pattern-dataset")
weather_df.head()

date-district-dataset 

district-feature-dataset
weather-pattern-dataset


,index,formal_date,quarter,date,day_of_week,month,Suspected_Local,Suspected_Foreign,temp,humidity,sun_hours,TotalInfected
0,1,2020-03-31,1,31,1,3,41,1,28.5,81.0,7.15,11
1,2,2020-03-31,1,31,1,3,1,0,28.5,76.0,8.50,15
2,3,2020-03-31,1,31,1,3,5,0,29.0,76.0,8.50,18
3,4,2020-03-31,1,31,1,3,2,0,28.5,75.0,8.00,0
4,5,2020-03-31,1,31,1,3,7,0,28.5,76.0,8.30,1


,ProvinceNumber,Province,Provincial Population,District Number,District,Elevation(m),Neighbour district numbers,Neighbour District Names,District Population,Population density /km2,No. of Police Stations,OIC division,Division Population,Unnamed: 13,Police station,Population
0,1,Western Province,"6,149,000",1.0,Colombo District,5.0,2.0,Gampaha,"2,448,000","3,502",48.0,Colombo Central,NaN,1.0,Pettah,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,3.0,Kalutara,NaN,NaN,NaN,NaN,NaN,2.0,Keselwatta,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,7.0,Ratnapura,NaN,NaN,NaN,NaN,NaN,3.0,Dam Street,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,8.0,Kegalle,NaN,NaN,NaN,NaN,NaN,4.0,Woulfendal,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,Fort,NaN


,District Number,Unnamed: 1,District,Elevation to City(m),Unnamed: 4,Unnamed: 5,Jan,Feb,March,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Avg
0,1.0,NaN,Colombo,5.0,NaN,NaN,27,28.0,28.5,28.5,28.5,28.5,28.0,28.0,27.5,27.5,27.0,27.5,27.88
1,2.0,NaN,Gampaha,15.0,NaN,NaN,27,28.0,28.5,28.5,29.0,28.5,28.0,27.5,28.0,27.5,28.0,28.0,28.04
2,3.0,NaN,Kalutara,10.0,NaN,NaN,26.5,27.5,28.0,29.0,29.0,28.5,28.5,27.5,28.0,27.5,27.0,27.0,27.83
3,4.0,NaN,Galle,8.0,NaN,NaN,27,27.0,28.5,28.5,28.0,28.0,27.5,27.5,27.5,27.5,27.0,27.0,27.58
4,5.0,NaN,Matara,10.0,NaN,NaN,26,26.0,28.5,28.5,28.0,28.0,27.5,27.5,27.5,28.5,27.0,27.0,27.50


In [11]:
date_dist_df['index'] = date_dist_df.iloc[:,0]
date_dist_df = date_dist_df.drop(['Unnamed: 0', 'ID', 'Date', 'District'],axis=1)
date_dist_df.head()

KeyError: "['Unnamed: 0' 'ID' 'Date' 'District'] not found in axis"

get province of given district



In [12]:
def pr_ds_mapper(x):
    pr_ds_dict = [1,1,1,2,2,2,3,3,4,4,4,5,5,6,6,6,7,7,8,8,9,9,9,9,9]
    return pr_ds_dict[int(x)-1]

date_dist_df['province'] = date_dist_df['index'].apply(lambda x: pr_ds_mapper(x))

coverted infected count of district
[date, district]

In [36]:
def get_date_infected(date):
    date_inf = [0]+list(date_dist_df[(date_dist_df.loc[:,'formal_date'] == date)]['TotalInfected'].values)
    if (len(date_inf) != 26):
        date_inf = [0,]*26
    return date_inf


In [37]:
def coverted_infected(date):

    srr = {1:[2,3,7,8],2:[1,3,8,17,18],3:[1,4,7],4:[3,5,7],5:[4,6,7],
           6:[5,7,13,15],7:[1,3,4,5,6,8,10,12,13],8:[1,2,7,9,10,17],
           9:[8,10,11,12,17],10:[7,8,9,12],11:[9,12,15,17,19,20],
           12:[7,9,10,11,15,13],13:[6,7,12,15],14:[15,16,20],15:[6,11,12,13,14,20],
           16:[14,19,20,22,25],17:[2,8,9,11,18,19],18:[2,17,19,24],
           19:[11,16,17,18,20,22,24],20:[5,14,15,16,19],21:[23],
           22:[16,19,24,25],23:[21,24,25],24:[18,19,22,23,25],25:[16,22,23,24]}

    infs_day_before = get_date_infected(date)

    inf_lis = []
    for i in range(1,26):
        nei_lis = srr[i]
        c = 0
        for j in nei_lis:
            c += infs_day_before[j]
        inf_lis.append(c)
    inf_ser = pd.Series(data=inf_lis,index=range(1,26))
    return inf_ser

In [38]:
res = coverted_infected('2020-03-31')

In [39]:
# dates = date_dist_df['formal_date'].unique().apply(lambda x: coverted_infected(x))
# res = np.vectorize(coverted_infected)(date_dist_df['formal_date'].unique())
def repeater(dates):
    for j in range(len(dates)):
        if(j == 0):
            fin_ser = coverted_infected(dates[j])
        else:
            fin_ser = pd.concat([fin_ser,coverted_infected(dates[j])])
    return fin_ser
res = repeater(date_dist_df['formal_date'].unique())

lockdown history melting


In [40]:
lock_df = pd.read_csv('../../../data/main_data/lockdown_history.csv')
lock_df.head()

,District Number,Unnamed: 1,District,1-Feb,2-Feb,3-Feb,4-Feb,1-Mar,2-Mar,20-Mar,...,22-Oct,23-Oct,24-Oct,25-Oct,26-Oct,27-Oct,28-Oct,29-Oct,30-Oct,31-Oct
0,1,NaN,Colombo,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.13,0.15,0.29,0.42,0.44,0.44,0.48,0.48,1.0,1.0
1,2,NaN,Gampaha,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0,1.0
2,3,NaN,Kalutara,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.00,0.12,0.12,0.12,0.12,0.12,0.19,0.19,1.0,1.0
3,4,NaN,Galle,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
4,5,NaN,Matara,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
